<a href="https://colab.research.google.com/github/AimjGuytidy/pytorch-on/blob/main/ml_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Simple ML Algo for Classification

### Implementing a perceptron learning algorithm

In [3]:
import numpy as np

In [2]:
class Perceptron:
  def __init__(self,lr=0.01,n_iter = 50,random_state = 1):
    self.lr = lr
    self.n_iter = n_iter
    self.random_state = random_state
  def fit(self,X,y):
    rgen = np.random.RandomState(self.random_state)
    self.w_ = rgen.normal(loc=0.0,scale = 0.01,size=X.shape[1])
    self.b_ = np.float_(0.)
    self.errors_ = []
    for _ in range(self.n_iter):
      errors = 0
      for xi,target in zip(X,y):
        update = self.lr*(target - self.predict(xi))
        self.w_ += update * xi
        self.b_ += update
        errors += int(update != 0.0)
      self.errors_.append(errors)
    return self
  def n_input(self,X):
    return np.dot(X,self.w_) + self.b_
  def predict(self,X):
    return np.where(self.n_input(X) >= 0.0,1,0)

0